1. Select runs that will contribute to the blend
2. Extract predictions on vsplit to use as features
3. Make sure that these features are aligned across the runs we chose
4. Use the same features to retrain on full-train and predict on test

5. Join predictions from various models and consider them to be the new features of our design matrix (our X variable)

6. Fit and predict a meta model using the new features

In [1]:
import numpy as np
import pandas as pd
import pymongo
from pymongo import MongoClient
import gridfs

In [2]:
client = MongoClient('localhost', 27017)
db = client.sacred
fs = gridfs.GridFS(db)
collection = db.runs

In [3]:
for doc in collection.find():
    print(doc['experiment']['repositories'])

[{'url': 'git@github.com:gilt/odsc-2018.git', 'commit': 'd08d8688a8c44e328ac5ded790e9cc9ff8e75596', 'dirty': True}, {'url': 'git@github.com:gilt/odsc-2018.git', 'commit': 'd08d8688a8c44e328ac5ded790e9cc9ff8e75596', 'dirty': True}, {'url': 'git@github.com:gilt/odsc-2018.git', 'commit': 'd08d8688a8c44e328ac5ded790e9cc9ff8e75596', 'dirty': True}]
[{'url': 'git@github.com:gilt/odsc-2018.git', 'commit': 'd08d8688a8c44e328ac5ded790e9cc9ff8e75596', 'dirty': True}, {'url': 'git@github.com:gilt/odsc-2018.git', 'commit': 'd08d8688a8c44e328ac5ded790e9cc9ff8e75596', 'dirty': True}, {'url': 'git@github.com:gilt/odsc-2018.git', 'commit': 'd08d8688a8c44e328ac5ded790e9cc9ff8e75596', 'dirty': True}]
[{'url': 'git@github.com:gilt/odsc-2018.git', 'commit': 'd08d8688a8c44e328ac5ded790e9cc9ff8e75596', 'dirty': True}, {'url': 'git@github.com:gilt/odsc-2018.git', 'commit': 'd08d8688a8c44e328ac5ded790e9cc9ff8e75596', 'dirty': True}, {'url': 'git@github.com:gilt/odsc-2018.git', 'commit': 'd08d8688a8c44e328ac5d

In [4]:
doc['experiment']['sources'][0][1]

ObjectId('5ad593caa2008c5fbde59854')

In [5]:
doc['result']

0.4798206278026906

In [6]:
q = collection.find()\
    .sort('result', pymongo.DESCENDING)\
    .limit(3)
    
for doc in q:
    print(doc['result'])

0.4798206278026906
0.452914798206278
0.4080717488789238


In [7]:
doc['artifacts']

[{'file_id': ObjectId('5ad593cba2008c5fbde5985a'), 'name': 'predictions'},
 {'file_id': ObjectId('5ad593cba2008c5fbde5985c'), 'name': 'submission'}]

In [8]:
o = doc['artifacts'][0]['file_id']
o

ObjectId('5ad593cba2008c5fbde5985a')

In [11]:
pd.read_csv(fs.get(o))

,PassengerId,pred_proba,Survived
0,129,0.512924,0
1,857,0.549045,1
2,637,0.434942,0
3,211,0.538840,1
4,117,0.449577,0
5,84,0.519449,1
6,330,0.367314,1
7,122,0.466266,0
8,181,0.527862,0
9,76,0.514629,0


In [7]:
np.load(fs.get(o))

array([0.51309009, 0.5413249 , 0.47851808, 0.5692588 , 0.64774206,
       0.51123312, 0.44150686, 0.50997385, 0.51242326, 0.61017481,
       0.51794069, 0.58053317, 0.51198627, 0.56792523, 0.30361622,
       0.51449272, 0.54350245, 0.46613098, 0.45275906, 0.55295553,
       0.53819804, 0.30361622, 0.51863009, 0.44047032, 0.30876852,
       0.535738  , 0.5       , 0.35495614, 0.51000835, 0.51089958,
       0.70004447, 0.44416392, 0.59173004, 0.46241193, 0.46063807,
       0.51089958, 0.54612017, 0.51069837, 0.51000835, 0.53819804,
       0.60763663, 0.50973233, 0.50983583, 0.51111237, 0.4670724 ,
       0.51794069, 0.57737405, 0.53457285, 0.54885655, 0.51794069,
       0.80030071, 0.51084217, 0.51195757, 0.51195757, 0.54227476,
       0.53218313, 0.51075012, 0.51311313, 0.54671548, 0.5461783 ,
       0.25442162, 0.51242326, 0.49795366, 0.44443662, 0.69978312,
       0.45564545, 0.52523975, 0.51973983, 0.51794069, 0.44853109,
       0.77290617, 0.51089958, 0.48906144, 0.51794069, 0.51000